In [24]:
from os import listdir
import uuid
from datetime import date
import datetime
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datetime import timedelta

In [6]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [2]:
def load_panel():
    social = pd.read_excel('/Users/cblanesg/misinformation_socialmedia/data/4-panel_data/newspapers/panel_newspapers.xlsx')
    return(social)

## Load Data

In [4]:
panel = load_panel()

## Apply Functions

In [7]:
## Prepare initial data
data = panel[panel['label_desinformacion'] != 'misleading']
data = data[~data['date_factcheck_facebook_clean'].isna()]
data['treatment'] = np.where(data['n_days_since_factcheck'] < 0, 0, 1)
data = data[['id_post', 'n_days_since_publicacion', 'treatment', 
             'date_factcheck_facebook_clean', 'n_days_since_factcheck', 
         'likes_facebook', 'shares_facebook', 'reactions_facebook', 'comments_facebook']]

In [15]:
panel_structure = []
for i in tqdm(data.id_post.unique()):
    id_data = data[data['id_post'] == i]
    df = pd.DataFrame({'n_days_since_factcheck':list(range(-16, 16))})
    df['id_post'] = i
    panel_structure.append(df)

100%|██████████| 16001/16001 [02:54<00:00, 91.79it/s]


In [16]:
df_structure = pd.concat(panel_structure)

In [36]:
input_intrapolation_temp  = pd.merge(df_structure, 
        data[['id_post', 
              'n_days_since_factcheck', 
              'likes_facebook', 
              'comments_facebook',
              'shares_facebook',
              'reactions_facebook']], 
        on = ['id_post', 'n_days_since_factcheck'], 
        how = 'outer')

In [37]:
input_intrapolation_temp = pd.merge(input_intrapolation_temp, 
        panel[['id_post', 
              'date_factcheck_facebook_clean', 
              'date_post_clean']].drop_duplicates(), 
        on = 'id_post', 
        how = 'left')

In [38]:
input_intrapolation_temp['date_timestep'] = input_intrapolation_temp.apply(lambda x: 
                                                                          x['date_factcheck_facebook_clean'] + timedelta(x['n_days_since_factcheck']) , axis = 1)

In [39]:
input_intrapolation_temp['n_days_since_publication'] = input_intrapolation_temp.apply(lambda x: 
                                                                          (x['date_timestep'] - x['date_post_clean']).days, axis = 1)

In [41]:
input_intrapolation_temp = input_intrapolation_temp[input_intrapolation_temp['n_days_since_publication'] >= 0] 

In [42]:
input_intrapolation_temp = input_intrapolation_temp[['id_post', 
                                                     'date_factcheck_facebook_clean', 
                                                     'date_post_clean', 
                                                     'date_timestep', 
                                                     'n_days_since_factcheck',
                                                     'n_days_since_publication', 
                                                     'likes_facebook', 
                                                     'comments_facebook',
                                                     'shares_facebook',
                                                     'reactions_facebook']].rename(columns = {'date_factcheck_facebook_clean':'date_factcheck', 
         'date_post_clean':'date_desinformacion'})

In [43]:
input_intrapolation_temp.to_excel('../../../data/4-panel_data/newspapers/input_intrapolation.xlsx')